In [ ]:
import os
import numpy as np
import math
import librosa
import json

In [ ]:
"""
    Script parameters
"""
target_amount_samples = 250
skipSamples = 0
target_dir='workspace/data/training/KEC'


"Persons"
speakers_in_room = 1

"ROOM"
randomize_room = False
# room generation will be laplace distributed in these intervalls [width, length, height]
normal_room_dim = [15, 20, 4]
room_dim_ranges = [[3, 30], [3, 30], [2.5, 5]]

# The amout Walls absorb Sound
normal_absorption = 1.2
absorption_range = [1, 2]

# the time it take until the signal drops by 60 dB
normal_rt60 = 0.5
rt60_range = [0.05, 0, 75]

"AUdio"
sampleRate = 16000
#total_length = 30
max_audio_length = 30
#min_audio_length = 5
min_characters = 20


# 'nice to have' for data generation
source_dataset = 'KEC - https://clarin.phonetik.uni-muenchen.de/BASRepository/index.php?target=Public/Corpora/KEC/KEC.1.php'


In [ ]:
def rec_filter(x):
    return True if 'rec' in x else False


def wav_filter(x):
    return True if '.wav' in x else False


def txt_filter(x):
    return True if '.TextGrid' in x else False


def as_filter(x):
    return True if 'AS' in x else False


def idx_filter_txt(x, y):
    return True if x[len(x) - 10] == y else False


def rec_filter(x):
    return True if 'rec' in x else False


def norm2(x):
    return np.sum(map(lambda y: y**2, x))


def distance(x,y):
    return norm2(x-y)

def avg(x):
    return sum(x)/len(x)

In [ ]:
def FileGeneratorsKEC():
    src = '/workspace/data/KEC'
    recFolders = list(filter(lambda f: rec_filter(f), os.listdir(src)))
    allWavfiles = []
    alltxtFiles = []
    for rec in recFolders:
        currentDir = src+"/"+rec

        wavFilesOfFolder = list(filter(lambda f: wav_filter(f), os.listdir(currentDir)))
        wavFilesOfFolder = list(map(lambda f: currentDir+"/"+f, wavFilesOfFolder))
        allWavfiles.extend(wavFilesOfFolder)

        txtFilesOfFolder = list(filter(lambda f: txt_filter(f), os.listdir(currentDir)))
        txtFilesOfFolder = list(map(lambda f: currentDir+"/"+f, txtFilesOfFolder))
        alltxtFiles.extend(txtFilesOfFolder)
    
    def wavGen():
        for file in allWavfiles:
            yield file
    def textGen():
        for file in alltxtFiles:
            yield file

    return wavGen, textGen

def VoiceLineGeneratorKEC(wavGen,textGen):
    # Todo
    pass


In [ ]:
def generate_room_characteristics():
    dims,rt60,absorption=0
    if randomize_room:
        dims: list(int) = [np.random.randint(room_dim_ranges[i, 0], room_dim_ranges[i, 1])
                           for i in range(len(room_dim_ranges[:]))]

        rt60: float = rt60_range[0] + \
            (rt60_range[1]-rt60_range[0])*np.random.random()

        absorption: float = absorption_range[0] + \
            (absorption_range[1]-absorption_range[0]) * np.random.random()

    else:
        dims = normal_room_dim
        rt60 = normal_rt60
        absorption = normal_absorption

    return dims, rt60, absorption

In [ ]:
def random_position_in_room(roomDims):
    x = np.random.random() * (roomDims[0]-1) + 0.5
    y = np.random.random() * (roomDims[1]-1) + 0.5
    z = 1.73
    return [x, y, z]


def positions_too_close(positions):
    for a in positions[1:]:
        for b in positions[1:]:
            if a == b:
                continue
            if distance(a, b) < 1:
                return True
    return False


# returns angle between two points in degrees
def get_angle(a, b):
    # turns clockwise angle into counter-clockwise
    def angle_trunc(a):
        while a < 0.0:
            a += math.pi * 2
        return a

    deltaY = b[1] - a[1]
    deltaX = b[0] - a[0]
    return math.degrees(angle_trunc(math.atan2(deltaY, deltaX)))


def transform_to_directivities(positions):
    dirs = []
    middle = [avg(positions[:, i]) for i in len(positions[0])]

    posi = positions[0]
    # Winkel aus Vogelperspeltive, in die Listnener guckt
    baseAngle = get_angle(posi[:2],middle[:2])

    # linkes und rechtes ohr des LIsteners
    dirs.append([(baseAngle + 90) % 360, 90])  # bug somewhere here
    dirs.append([(baseAngle + 270) % 360, 90])

    for pos in positions[2:]:
        dirs.append([get_angle(pos,posi), 90])

    return dirs


def random_persons_in_room(roomDims):
    def pos_in_room(count):
        pos = []
        for i in range(count):
            pos.append(random_position_in_room(roomDims))
        return pos

    # do while: erzeuge solange bis gültiges ergebnis
    positions = pos_in_room(speakers_in_room)
    while(positions_too_close(positions)):
        positions = pos_in_room(speakers_in_room)
    positions[0] = positions[1]

    return positions, transform_to_directivities(positions)

In [ ]:
def loadWavFile(path, offset=0, duration=None):
    wav, sr = librosa.load(path, sr=sampleRate, offset=offset,
                           duration=duration, mono=True)

    wav = librosa.util.normalize(wav)
    return wav


In [ ]:


def get_words_with_timestamps(timestamps, j, words, word_ts):
    max_as = timestamps[j][1]
    min_as = timestamps[j][0]

    words2 = get_words(words, word_ts, [min_as, max_as])

    return words2


# retrives words with relative timestamp
def get_words(words, timestamps, interval):

    start = [(idx, time) for idx, time in enumerate(
        timestamps) if time[0] == interval[0]][0]
    end = [(idx, time) for idx, time in enumerate(
        timestamps) if time[1] == interval[1]][0]

    start_time = timestamps[start[0]][0]

    words = words[start[0]:end[0]+1]
    timestamps = timestamps[start[0]:end[0]+1]

    for idx, w in enumerate(words):
        words[idx] = [w, [timestamps[idx][0] -
                          start_time, timestamps[idx][1] - start_time]]
    return words

# splits text file in parts


def split_sentence(textGrid):

    helper = 0
    sentences, timestamp = [], []
    words, w_timestamp = [], []
    sentence = ''
    sentence_start = 0
    counter = 0

    for t in textGrid[0]:
        if helper > t.maxTime:
            break

        text = t.mark
        maxTime = t.maxTime
        minTime = t.minTime
        helper = maxTime
        sentence += ' ' + text
        words.append(text)
        w_timestamp.append([minTime, maxTime])
        counter += 1

        if '<P>' in text and maxTime - sentence_start < max_audio_length and maxTime - sentence_start >= 5 and len(sentence) > min_characters:
            sentences.append(sentence)
            sentence = ''
            timestamp.append([sentence_start, maxTime])
            sentence_start = maxTime

        elif '<P>' in text:
            sentence = ''
            sentence_start = maxTime
    return sentences, timestamp, words, w_timestamp


In [ ]:
# sampleNr: int, speakerIdsList:int[n], positionList:float[n+2], directions:float[n+2], timeframes:floats[n][2]
def createJsonData(sampleNr, speakerIdsList, positionList, directions, timeframes):
    speakers = []
    for i in range(len(speakerIdsList)):
        speaker = {
            'id': speakerIdsList[i],
            'position': positionList[i+2],
            'direction': directions[i+2],
            'startTime': timeframes[i, 0],
            'duration': timeframes[i, 1]
        }
        speakers.append(speaker)
        
    return {
        'comment': 'direction: [azimuth, colatitude]',
        'sample': {
            'id': sampleNr,
            'speakers': speakers,
            'listener': {
                'position': positionList[0],
                'directionLeft': directions[0],
                'directionRight': directions[1]

            },
            'source': source_dataset
        }
    }


def generate():
    global skipSamples
    wavGen, txtGen = FileGeneratorsKEC()
    for wav in wavGen():
        if(skipSamples > 0):
            skipSamples -= 1
            continue

        loadWavFile(wav, 0, 10)
        sentences_as, timestamps_as, word_as, word_ts_as = split_sentence(txtGen().__next__())
        print(sentences_as)
        print(timestamps_as),
        print(word_as)
        print(word_ts_as)
        break


if __name__ == '__main__':
    generate()
